<a href="https://colab.research.google.com/github/tuhinmallick/AI-for-Fashion/blob/main/GPU_Benchmarking_for_LoRA%2C_QLoRA_Fine_tuning%2C_and_Inference_with_and_without_4_bit_Quantization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*More details in this article: [GPU Benchmarking: What Is the Best GPU for LoRA, QLoRA, and Inference?](https://newsletter.kaitchup.com/p/gpu-benchmarking-what-is-the-best)*

This notebook benchmarks your hardware configuration for LoRA, QLoRA fine-tuning, and inference with and without 4-bit Quantization.

It uses Hugging Face Transformers, TRL, and PEFT for the fine-tuning part. For the inference part, the benchmarking is done through optimum-benchmark.

For fine-tuning, I use the training time returned by the logs as the benchmarking metric.

For inference, optimum benchmark produces a CSV file containing a lot of information that you can use as benchmarking metrics (memory consumption, latency, etc.)

*Note: This notebook only works with Ampere and more recent NVIDIA GPUs*

In [ ]:
!pip install optimum-benchmark bitsandbytes datasets peft trl
!pip install nvidia-ml-py


[notice] A new release of pip is available: 23.3.1 -> 24.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.1.1
[notice] To update, run: python -m pip install --upgrade pip


Method for LoRA and QLoRA fine-tuning:

In [ ]:
import torch, os, multiprocessing
from datasets import load_dataset
from peft import LoraConfig, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)
from trl import SFTTrainer, SFTConfig

#use bf16 and FlashAttention if supported
if torch.cuda.is_bf16_supported():
  os.system('pip install flash_attn')
  compute_dtype = torch.bfloat16
  attn_implementation = 'flash_attention_2'
else:
  compute_dtype = torch.float16
  attn_implementation = 'sdpa'


def LoRA(model_id, q=False):
  model_name =  model_id.split('/')[1]
  #Tokenizer
  tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
  tokenizer.pad_token = tokenizer.eos_token
  tokenizer.padding_side = 'left'

  ds = load_dataset("timdettmers/openassistant-guanaco")
  #Add the EOS token
  def process(row):
      row["text"] = row["text"]+"<|end_of_text|>"
      return row

  ds = ds.map(
      process,
      num_proc= multiprocessing.cpu_count(),
      load_from_cache_file=False,
  )
  if q:
    bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=compute_dtype,
            bnb_4bit_use_double_quant=True,
    )
    model = AutoModelForCausalLM.from_pretrained(
              model_id, quantization_config=bnb_config, device_map={"": 0}, attn_implementation=attn_implementation
    )
    model = prepare_model_for_kbit_training(model, gradient_checkpointing_kwargs={'use_reentrant':True})
    output_dir = model_name+"_QLoRA"
  else:
    model = AutoModelForCausalLM.from_pretrained(
            model_id, device_map={"": 0}, attn_implementation=attn_implementation, torch_dtype=compute_dtype
    )
    model.gradient_checkpointing_enable(gradient_checkpointing_kwargs={'use_reentrant':True})
    output_dir = model_name+"_LoRA"

  peft_config = LoraConfig(
          lora_alpha=16,
          lora_dropout=0.05,
          r=16,
          bias="none",
          task_type="CAUSAL_LM",
          target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"]
  )


  training_arguments = SFTConfig(
          output_dir="./"+output_dir,
          optim="adamw_8bit",
          per_device_train_batch_size=2,
          gradient_accumulation_steps=8,
          log_level="debug",
          save_strategy="epoch",
          logging_steps=10,
          learning_rate=1e-4,
          fp16 = not torch.cuda.is_bf16_supported(),
          bf16 = torch.cuda.is_bf16_supported(),
          num_train_epochs=1,
          warmup_ratio=0.1,
          lr_scheduler_type="linear",
          dataset_text_field="text",
          max_seq_length=512,
  )

  trainer = SFTTrainer(
          model=model,
          train_dataset=ds['train'],
          peft_config=peft_config,
          tokenizer=tokenizer,
          args=training_arguments,
  )

  trainer.train()
  del model
  del trainer
  gc.collect()
  torch.cuda.empty_cache()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 12.6 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 22.4 MB/s eta 0:00:00
  Created wheel for flash_attn: filename=flash_attn-2.5.9.post1-cp310-cp310-linux_x86_64.whl size=121711011 sha256=c55cb075a15591ebea0e8df05daea937ae206afd67ad1c50524a36f37cbd8d1d
  Stored in directory: /root/.cache/pip/wheels/cc/ad/f6/7ccf0238790d6346e9fe622923a76ec218e890d356b9a2754a
Successfully built flash_attn



[notice] A new release of pip is available: 23.3.1 -> 24.1.1
[notice] To update, run: python -m pip install --upgrade pip


Method for benchmarking the inference:

Note:
- Change the input shape for different batch size and sequence length
- Change "model_names" to add the models you wish to benchmark

In [ ]:
from optimum_benchmark import Benchmark, BenchmarkConfig, TorchrunConfig, InferenceConfig, TrainingConfig, PyTorchConfig
from optimum_benchmark.logging_utils import setup_logging
from transformers import set_seed
import gc
set_seed(1234)

model_names = ["meta-llama/Meta-Llama-3-8B"]

def inference_bench(model_id, quant=False):
    model_name = model_id.split('/')[1]
    launcher_config = TorchrunConfig(nproc_per_node=1)
    input_shapes = {"batch_size": 1, "num_choices": 1, "sequence_length": 200}

    scenario_config = InferenceConfig(latency=True, memory=True, input_shapes=input_shapes)
    if quant:
      name = "benchmark_inference_report_quant"+model_name+".csv"
      quantization_scheme = 'bnb'
      quantization_config = {
                              "bnb_4bit_compute_dtype": "float16",
                              "bnb_4bit_quant_type": "nf4",
                              "bnb_4bit_use_double_quant": True,
                              "llm_int8_enable_fp32_cpu_offload": False,
                              "llm_int8_has_fp16_weight": False,
                              "llm_int8_threshold": 6.0,
                              "load_in_4bit": True,
                              "load_in_8bit": False,
                            }
      backend_config = PyTorchConfig(model=model_id, quantization_scheme=quantization_scheme, torch_dtype="bfloat16", quantization_config=quantization_config, device="cuda", device_ids="0", no_weights=False)
    else:
      name = "benchmark_inference_report_"+model_name+".csv"
      backend_config = PyTorchConfig(model=model_id, device="cuda", torch_dtype="bfloat16", device_ids="0", no_weights=False)
    benchmark_config = BenchmarkConfig(
        name="pytorch_"+model_name,
        scenario=scenario_config,
        launcher=launcher_config,
        backend=backend_config,
    )
    benchmark_report = Benchmark.launch(benchmark_config)


    benchmark_report.log()
    benchmark_config.to_dict()
    benchmark_report.save_csv(name)


for m in model_names:
  LoRA(m, True) #QLoRA
  LoRA(m) #LoRA

  inference_bench(m) #Inference, not quantized (bfloat16)
  inference_bench(m, True) #Inference, quantized


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

[RANK-PROCESS-0][2024-07-05 11:42:26,370][inference][INFO] - 	+ Running Text Generation memory tracking
[RANK-PROCESS-0][2024-07-05 11:42:26,371][memory][INFO] - 	+ Tracking RAM memory of process [6131]
[RANK-PROCESS-0][2024-07-05 11:42:26,372][memory][INFO] - 	+ Tracking VRAM memory of CUDA devices [0]
[RANK-PROCESS-0][2024-07-05 11:42:26,372][memory][INFO] - 	+ Tracking Allocated/Reserved memory of 1 Pytorch CUDA devices
[RANK-PROCESS-0][2024-07-05 11:42:43,185][memory][INFO] - 		+ prefill memory:
[RANK-PROCESS-0][2024-07-05 11:42:43,186][memory][INFO] - 			- max RAM: 1326.043136 (MB)
[RANK-PROCESS-0][2024-07-05 11:42:43,187][memory][INFO] - 			- max global VRAM: 17474.781184 (MB)
[RANK-PROCESS-0][2024-07-05 11:42:43,187][memory][INFO] - 			- max process VRAM: 0.000000 (MB)
[RANK-PROCESS-0][2024-07-05 11:42:43,188][memory][INFO] - 			- max reserved memory: 16443.768832 (MB)
[RANK-PROCESS-0][2024-07-05 11:42:43,188][memory][INFO] - 			- max allocated memory: 16251.908096 (MB)
[RANK-PR

Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.23it/s]


[RANK-PROCESS-0][2024-07-05 11:43:54,951][pytorch][INFO] - 	+ Turning on model's eval mode
[RANK-PROCESS-0][2024-07-05 11:43:54,955][inference][INFO] - Allocating inference scenario
[RANK-PROCESS-0][2024-07-05 11:43:54,956][inference][INFO] - 	+ Creating input generator
[RANK-PROCESS-0][2024-07-05 11:43:54,956][inference][INFO] - 	+ Generating Text Generation inputs
[RANK-PROCESS-0][2024-07-05 11:43:54,957][inference][INFO] - 	+ Updating Text Generation kwargs with default values
[RANK-PROCESS-0][2024-07-05 11:43:54,957][inference][INFO] - 	+ Initializing Text Generation report
[RANK-PROCESS-0][2024-07-05 11:43:54,958][inference][INFO] - 	+ Preparing inputs for Inference
[RANK-PROCESS-0][2024-07-05 11:43:54,997][inference][INFO] - 	+ Preparing backend for Inference
[RANK-PROCESS-0][2024-07-05 11:43:54,998][inference][INFO] - 	+ Warming up backend for Text Generation


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


[RANK-PROCESS-0][2024-07-05 11:44:03,154][inference][INFO] - 	+ Running Text Generation memory tracking
[RANK-PROCESS-0][2024-07-05 11:44:03,155][memory][INFO] - 	+ Tracking RAM memory of process [6510]
[RANK-PROCESS-0][2024-07-05 11:44:03,156][memory][INFO] - 	+ Tracking VRAM memory of CUDA devices [0]
[RANK-PROCESS-0][2024-07-05 11:44:03,156][memory][INFO] - 	+ Tracking Allocated/Reserved memory of 1 Pytorch CUDA devices
[RANK-PROCESS-0][2024-07-05 11:44:21,035][memory][INFO] - 		+ prefill memory:
[RANK-PROCESS-0][2024-07-05 11:44:21,036][memory][INFO] - 			- max RAM: 1057.619968 (MB)
[RANK-PROCESS-0][2024-07-05 11:44:21,037][memory][INFO] - 			- max global VRAM: 7100.170240 (MB)
[RANK-PROCESS-0][2024-07-05 11:44:21,038][memory][INFO] - 			- max process VRAM: 0.000000 (MB)
[RANK-PROCESS-0][2024-07-05 11:44:21,038][memory][INFO] - 			- max reserved memory: 6069.157888 (MB)
[RANK-PROCESS-0][2024-07-05 11:44:21,039][memory][INFO] - 			- max allocated memory: 5893.362688 (MB)
[RANK-PROCE